In [1]:
import torch
import csv
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from tqdm import tqdm

In [142]:
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM

In [135]:
import nltk
from nltk.tokenize import sent_tokenize

In [34]:
def get_encode_data(tokenizer, sentences, max_length=256):
    encoded_inputs = tokenizer(sentences, padding=True, truncation=True, max_length=max_length)
    input_ids = torch.tensor(encoded_inputs['input_ids'])
    attention_masks = torch.tensor(encoded_inputs['attention_mask'])

    return (input_ids, attention_masks)

In [35]:
def check_ad(text, tokenizer, model, max_length=256) :
    encode_data = get_encode_data(tokenizer, [text], max_length=max_length)
    output = model(encode_data[0], encode_data[1])
    _, predicted = torch.max(output.logits, dim=1)
    predicted = predicted.to(torch.device('cuda'))
    predicted = predicted.item()
    
    return predicted

In [136]:
def get_model(model_name, device, tokenizer=None) :

    print(model_name)

    if tokenizer is None :
      tokenizer = AutoTokenizer.from_pretrained(model_name)

    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


    # parallelization
    if torch.cuda.device_count() > 1:
        print(f'Using {torch.cuda.device_count()} GPUs.')

        model = torch.nn.DataParallel(model)

    return model.to(device), tokenizer

In [137]:
def summarize(input_text, tokenizer, model, device, max_tokens=200, min_tokens=100, num_beams=2) :
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate Summary Text Ids
    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_new_tokens=max_tokens,
        min_new_tokens=min_tokens,
        no_repeat_ngram_size=3,
        num_beams=num_beams,
    )

    response = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

    #print(len(input_text), len(response))

    return response

In [138]:
def seperate_sentence(sentence, chunk) :
    ss = sent_tokenize(sentence)
    
    sentences = []
    merged = ""

    for s in ss :
        if len(merged) + len(s) <= chunk :
            merged = merged + s
        else :
            sentences.append(merged)
            merged = ""
        
    if len(merged) > 256 :
        sentences.append(merged)
    elif len(merged) > 0 :
        sentences[-1] = sentences[-1] + merged
        
    return sentences

In [139]:
def summarize_long_sentence(sentence, chunk, tokenizer, model, device, max_tokens, min_tokens, num_beams, verify=False) :
    
    ss = seperate_sentence(sentence, chunk)
    summaries = []
    
    for s in ss :
        if verify == True :
            print(s)
        
        summary = summarize(s, tokenizer, model.module, device, max_tokens, min_tokens, num_beams)
        summaries.append(summary)

    return summaries

In [3]:
model_name = 'klue/roberta-base'
b_model=128
length=256
code_file = '/home/osung/work/lm_classification/klue_roberta-base_ad_train.tsv_ad_code.csv'

In [161]:
pretrained_file = '/home/osung/work/lm_classification/klue_roberta-base_ad_total.tsv_ad_b128_e10_ml256_20240814_135702.pth'
num_labels = 2

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('available device: ', device)
else:
    device = torch.device("cpu")
    print('available device: ', device)

available device:  cuda


In [162]:
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False,)

In [7]:
pretrained_model_config = AutoConfig.from_pretrained(model_name)

In [8]:
pretrained_model_config.num_labels = num_labels

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=pretrained_model_config,
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model = torch.nn.DataParallel(model)

In [11]:
model = model.to(device)

In [12]:
codes = {}
codes[True] = 1
codes[False] = 0

In [163]:
loaded_state_dict = torch.load(pretrained_file)

In [164]:
model.load_state_dict(loaded_state_dict)

<All keys matched successfully>

In [124]:
text = "“교육·교통·문화 모두 갖춘 강남 최고의 입지” 지상 38층·지하 3층에 15개동 1,612세대 중 일반분양 213세대 한강 조망권·주거환경 등 호평 모델하우스 수요자 발길 줄이어 9·1 부동산대책 발표 후 강남 재건축단지의 몸값이 나날이 높아지고 있는 가운데 서울 강남권 재건축 아파트 중 신반포1차를 재건축한 `아크로리버 파크 2회차'가 9·1 대책 발표 후 가장 먼저 분양에 나서면서 고액 자산가들의 관심이 집중되고 있다. 강남 최고 부촌 아파트를 넘보는 대림산업의 `아크로리버 파크 2회차'가 지난 19일 견본주택의 문을 열고 본격 분양에 들어갔다. 단지는 지하 3층~지상 38층, 15개 동, 전용면적 59~164㎡로 전체 1,612세대 중 213세대가 일반 분양된다. 세부면적을 살펴보면, △59㎡ 40세대 △84㎡ 118세대 △112㎡ 16세대 △129㎡ 33세대 △164㎡ 6세대로 중소형이 74%를 차지한다. 입주시기는 2016년 8월 예정이다. ■반포지구 재건축단지 중 최고 층수로 지어져 … 반포지역 프리미엄 톡톡=반포 `아크로리버 파크'는 교육, 교통, 문화, 편의시설 등 완벽한 주거환경을 갖춰 대표 부촌으로 자리를 굳히고 있는 반포의 중심에 위치하는 만큼 지역 프리미엄을 톡톡히 누릴 수 있는 곳이다. 또한 이 단지는 강남 한강변에 10년 만에 공급된 아파트인데다 강남 한강변에서는 최초로 특별건축구역으로 지정돼 최고 38층 초고층으로 건설된다. 다른 반포지구 재건축단지 최고 층수는 34~35층으로 향후 반포동 랜드마크 단지로서의 역할이 기대된다. ■한강 조망 가능한 `고층 중심', 대리석 마감 및 독일산 주방가구로 `상품 업그레이드'=아크로리버 파크 2회차의 분양가는 현재 서초구에서 가장 고가 아파트이며 입주 5년차를 맞은 `래미안퍼스티지'의 전용 59㎡가 8억9,000만~10억5,000만원 사이에 거래되는 것과 비교해 비슷한 수준에 책정됐다. 아크로리버 파크 2회차 총 분양가는 전용 59㎡가 8억4,900만~10억5,000만원, 전용 84㎡는 11억8,000만~15억4,500만원선으로 책정됐다. 전용 112㎡는 16억3,000만~20억1,000만원, 전용 129㎡ 18억~21억4,000만원, 전용 164㎡는 21억8,000만~23억9,000만원이다. 계약조건은 계약금 10%, 중도금 이자 후불제가 적용된다. ■최고 부촌에 지어지는 최고가 명품 아파트답게 고급화된 설계 눈길=반포 `아크로리버 파크'가 강남 노른자위에 지어지는 최고가 명품 단지인 만큼 2회차 물량에도 최고급 설계가 적용된다. 우선 동 배치를 오픈 뷰(Open view)와 59m의 동 간 이격 거리로 실현해 탁 트인 조망권과 풍부한 일조권을 확보했다. 또한 단지 내 6개 테마정원뿐만 아니라 서측공원과 보행로를 따라 한강시민공원 반포지구까지 연결돼 있어 쾌적한 주거환경을 갖췄으며, 단지 내에서도 메인 커뮤니티 및 지하철 접근성이 우수한 것도 특징으로 꼽힌다. 견본주택은 서울 강남구 신사동 631 도산공원 사거리에서 성수대교 남단 교차로 방면 우측에 있다."

In [125]:
check_ad(text, tokenizer, model)

0

In [126]:
text = '인천 부동산시장 양극화 인천지역 아파트 청약시장이 과열현상까지 보이고 있는 반면, 기존 주택시장은 침체의 늪에서 빠져 나오지 못하는 등 양극화 현상이 심화되고 있다. 11일 지역 건설·부동산업계에 따르면 최근 경제자유구역을 중심으로 아파트 청약열풍이 불고 있지만 기존 주택시장은 매매가 끊긴 채 장기 침체에서 벗어나지 못하고 있다. 지난달과 이달 중 경제자유구역 청라지구에 공급된 아파트단지 3곳 청약에서 최고 7대 1의 경쟁률을 기록하며 아파트단지 3곳 모든 평형들이 1순위에서 마감됐다. 특히 송도국제도시 아파트 분양을 위해 지난 8일 문을 연 P건설 모델하우스에는 예비청약자들이 하루평균 1만명씩 몰려 북새통을 이루며 지난 2006년 나타났던 청약열풍이 재현될 조짐까지 보이고 있다. 그러나 기존 주택시장은 지난 2007년부터 이어진 침체의 늪에서 좀처럼 벗어나지 못하고 있다. 지난 2006년 분양돼 지난해 8월 입주한 남동구 A아파트는 700만~800만원대인 낮은 분양가 수준에서도 거래가 이뤄지지 않은 채 전세수요만 조금 있는 실정이다. 연수구 B아파트도 지난 2007년초 가격 대비 최고 1억원까지 떨어진 2억6천~2억7천만원대 매물들이 쏟아지고 있지만 주인을 찾기가 쉽지 않다. 이 같은 양극화 현상은 분양가상한제가 적용된 싼 분양가 아파트 청약에는 투자자와 실수요자들이 입주 시기인 2~3년 이후를 기대하며 몰리는 반면, 기존 시장에는 침체된 현재의 경기가 그대로 반영되고 있는데 따른 것으로 분석된다.'

In [127]:
check_ad(text, tokenizer, model)

1

In [128]:
text = '고분양가 그대로 두고 미분양 해소! 바보 아냐? "건설사들이 \'같이 죽든지 아니면 미분양을 해소시켜 달라\'고 하니 정부에서 나선 거다. 그런데 건설사가 \'(미분양) 책임 없다\'며 비싼 분양가를 안 내리니 미분양 해소가 되겠느냐." 지난 18일 경기 용인시 수지구 성복동에서 만난 공인중개사 김정수(가명)씨가 한숨 쉬듯 말했다. 지난 2월 12일 정부가 미분양 해소를 위해 내놓은 양도소득세 면제·감면 조치의 효과에 대해 묻자 돌아온 답이었다. 김씨의 공인중개사무소 인근에는 2·12 조치의 대표적인 수혜단지로 언론에 보도된 \'용인 성복 힐스테이트\'가 있다. 이 아파트를 두고 그는 "완전히 \'1+1\'"이라며 "힐스테이트 분양가격으로 인근 아파트 2채를 살 수 있다, 누가 사겠느냐?"고 지적했다. 언론의 섣부른 보도와는 달리 고분양가 탓에 미분양 해소라는 정책적 효과를 기대할 수 없다는 게 김씨의 생각이다. 또한 그는 양도세 감면·면제 혜택을 보지 못하는 계약자들이 아파트 분양권을 싸게 내놓아, 미분양 해소는 더욱 어렵다고 덧붙였다. 이는 2·12 조치의 다른 수혜단지의 사정도 마찬가지였다. 모델하우스 북적? 빈 책상에 앉은 분양 상담사들 이날 오전에 찾은 경기 성남시 분당구 구미동 \'용인 성복 힐스테이트\' 모델하우스는 적막감이 감돌았다. 상담사 3명이 빈 책상에 앉아 방문객을 기다리고 있었다. 이곳의 분위기는 "용인 지역 미분양 아파트는 앞으로 5년간 양도세가 면제되기 때문에, 2·12 조치 발표 이후 방문객과 문의가 폭주하면서 미분양이 빠르게 소진되고 있다"는 일부 언론 보도와는 거리가 멀었다. 반면 현대건설 분양팀 관계자는 "2·12 조치 이후 하루 10명이던 방문객이 200여 명으로 늘었다"며 "가계약하겠다는 40~50명을 확보했다"고 말했다. 현재 모두 2157세대인 이 아파트단지의 미분양 물량은 700여 세대다. 이 관계자는 이어 "현재 양도세 감면 조치가 어느 정도 효과가 있을지는 지켜봐야 한다"며 "경기침체가 가속화되고 있는 상황에서 강남이 오르지 않는 이상, 용인도 가격이 상승하기 어렵다"고 전했다. "비싼 미분양 아파트 사는 건 바보 짓" 성복지구 아파트 건설 현장 인근에서 만난 공인중개사들은 상황을 더 비관적으로 보고 있었다. "미분양 원인은 다 있다, 힐스테이트는 분양가가 너무 비싸다"며 "건설사가 분양가격을 내리거나 부동산 시장이 크게 살아나지 않는 한 미분양 해소는 어렵다"고 전했다. 힐스테이트 199㎡(60평)형의 경우, 발코니 확장과 장식가구 등 옵션을 모두 포함한 분양가는 10억387만원이다. 3.3㎡(1평)당 분양가는 1673만원. 분양가의 60%에 이르는 중도금의 이자까지 더하면 3.3㎡당 분양가가 1700만원을 넘는다는 게 공인중개사들의 분석이다. 이에 비해 힐스테이트 인근의 엘지빌리지 204㎡(62평) 가격은 6억5천만원으로 3.3㎡당 1000만원 안팎이다. L공인중개사 대표는 "나중에 가격이 오를지 모르지만, 현재로선 힐스테이트 미분양 아파트를 사는 것은 \'바보짓\'이다, 분양가보다 7천만원 떨어진 분양권도 나왔지만 사는 사람이 없다"고 지적했다. 이어 그는 "용인 수지 동일하이빌 아파트의 경우 분양가를 10% 인하해서 그나마 미분양 물량이 소진되고 있다"며 "건설사들이 충분히 분양가를 인하할 수 있지만, 안 하는 건 사기"라고 지적했다. 힐스테이트의 가계약자도 가격이 부담되긴 마찬가지다. 모델하우스에서 만난 한 가계약자는 "분당에 살고 있는데 이사를 오려고 한다"면서 "가격이 많이 떨어진 분당 집을 팔고 여기로 오기엔 가격 차이가 너무 크다, 계약을 고민하고 있다"고 전했다. 양도세 완화 수혜단지? 혜택 받는 아파트 숫자 많지 않아 지난 16일 찾은 경기 고양시 일산동구 식사지구에 건설 중인 일산자이의 모델하우스 분위기도 미분양 해소와는 거리가 멀었다. 이곳 역시 언론에 의해 대표적인 양도세 완화 수혜단지로 꼽혔지만, 실제로는 양도세 완화 대상인 중소형 아파트가 많지 않다. 과밀억제권역에 포함되는 고양시에서는 공급면적이 149㎡(45평) 이내인 미분양과 신규 분양 아파트에 한해서 양도세가 50% 감면된다. 모두 4683세대(112~276㎡)인 일산 자이에서 149㎡ 이하인 아파트(주상 복합 제외)는 559세대에 불과하다. 일산자이 분양사무소 관계자는 "전체의 20% 정도 미분양이 남았는데, 대부분 대형으로, 이번 조치로 직접적인 수혜는 없다"면서도 "대신 2·12 조치를 시작으로 앞으로 많은 규제가 풀릴 것으로 생각하는 투자자들이 움직이고 있다, 주말 동안 70여 세대가 계약됐다"고 전했다. 하지만 이날 모델하우스에서 만난 사람들은 고분양가 탓에 선뜻 계약 결정을 하지 못했다. 일산 자이의 분양가는 3.3㎡당 1450만원대다. 현재 \'계약금 3500만원\', \'중도금 무이자 융자\', \'에어컨 제외한 옵션 무상제공\' 등을 내걸었지만, 아직 기대만큼 큰 반응은 없다. 모델하우스 방문객 이현진(가명·37)씨는 "식사지구 맞은편에 2년 전 분양한 아파트의 분양가격은 3.3㎡당 800만원대였고, 지금은 900만원대"라며 "다들 (시세차익을) 노리고 투자하는 것이지만, 분양가가 비싼 건 사실"이라고 말했다. "양도세 완화 조치로 미분양 해소되지 않는다" 모델하우스 앞 \'떴다방(이동식 중개업소)\'이나 인근 공인중개사무소에서는 "미분양 해소 효과는 거의 없을 것"이라고 전망했다. 한 공인중개사는 "일산 자이에 관심 있는 중개업자는 많지 않다"며 "인근의 3.3㎡당 천만원인 아파트도 잘 거래가 안 되는데, 누가 평당 1400만원대 아파트를 사겠느냐"고 반문했다. \'떴다방\' 업자 김미진(가명)씨는 "(양도세 감면에) 해당하는 아파트도 많지 않을뿐더러, 시세차익을 기대하기 어려우니 양도세 감면 효과는 없고, 미분양 해소도 쉽지 않을 것"이라고 말했다. 그의 말이다. "2·12 이전 계약자는 그 이후 계약자보다 손해를 본다고 생각하기 때문에 프리미엄을 받지 않고 팔 것이다. 집값이 언제 오를지 모르는 상황에서 시세차익을 기대하기보다는 싼 기존 분양권을 사는 사람이 많아진다. 누가 미분양을 사겠느냐. 미분양은 해소되지 않는다."'

In [129]:
check_ad(text, tokenizer, model)

1

In [33]:
test_df = pd.read_csv('/home/osung/data/korean/modu/json/combined_news.tsv', sep='\t')

In [40]:
ad_df = test_df[test_df['text'].apply(lambda x: all(keyword in x for keyword in ['아파트', '분양', '건설', '분양가', '모델하우스']))]

In [84]:
def predict_ad(text, model, tokenizer) :
    #print(text)
    encode_data = get_encode_data(tokenizer, [text], max_length=256)
    output = model(encode_data[0], encode_data[1])
    _, predicted = torch.max(output.logits, dim=1)
    predicted = predicted.item()
    
    if predicted : 
        return True
    else :
        return False

In [144]:
max_tokens = 200
min_tokens = 100
num_beams = 2

In [82]:
ad_df.text

1406       고분양가 그대로 두고 미분양 해소! 바보 아냐? "건설사들이 '같이 죽든지 아니면 ...
2245       아파트 계약자들과 시공사 만남이 '화기애애'? 건설업체가 변하고 있다. 장기침체로 ...
2576       도심 재개발 원주민 정든 마을 내쫓길 처지 재개발 보상가 문제로 일부 주민들의 거센...
9456       "서해 조망권이라더니 민둥산 조망권이었나?" "강남에 지은 곳보다 건축비가 더 들었...
14437      대법, 분양률 높이려 과장 광고한 건설사 제동 신규 아파트 인근에 전철역이 들어선다...
                                 ...                        
3563770    ‘김해율하 더스카이시티’ 특별공급 신청 몰려...6일 1순위 청약 시작 두산건설·코...
3571931    분양가상한제적용 회천 센트럴 아리스타 아파트, 단지내상업시설 관심 한국부동산신탁이 ...
3572202    웨이브파크 앞 황금 입지… 해양레포츠 특화 복합스트리트몰 ‘시화MTV 보니타가’ 분...
3572646    삽질 대신 클릭에 진심인 건설회사들… 메타버스로 출근하는 ‘제한살’ 직원들 ●메타버...
3609046    ‘넓고 쾌적하게 살자’ 중산층 수요 많아 … 신도시 인프라 누리면서 전원생활 가능해...
Name: text, Length: 2182, dtype: object

In [69]:
predict_ad(ad_df.iloc[3].text, model, tokenizer)

"서해 조망권이라더니 민둥산 조망권이었나?" "강남에 지은 곳보다 건축비가 더 들었는데 자재는 좋지 않은 것 같아요. 조망권을 강조했는데 아파트에서 보이는 건 민둥산뿐이에요." "토지를 3.3㎡당 191만 원에 샀다는데 분양가는 평균 1070만 원이에요. 인근에 지금 분양하는 곳보다 200만 원 비싸요." 26일 인천 영종도에 들어선 GS건설의 영종자이 아파트 앞에서 이종혁(가명·53)씨와 최진수(가명·56)씨는 한숨 섞인 대화를 이어나갔다. 이들은 29일 이 아파트에 입주할 예정이지만 설렘은 사라진 지 오래다. 최씨는 "국내 굴지의 건설사 이름을 믿고 아파트를 분양받았는데 알고 보니 건설사는 폭리를 취했고 우리는 속았다"며 "GS건설의 이러한 비윤리적인 모습에 말이 나오지 않는다"고 전했다. 처음에 입주 예정자들은 부실시공과 '민둥산 조망권'을 문제 삼았다. 이후 택지비와 건축비가 실제보다 과도하게 책정됐음을 보여주는 자료를 발견한 입주 예정자들은 시행사와 건설사에 분양가 할인을 요구하고 나섰다. 그러나 GS건설은 "일부 하자는 입주 때까지 처리하겠다"면서 "부실시공이나 분양가 폭리는 없다"고 강조했다. 서해가 보이는 명품 아파트라는 말에 샀더니... '무덤 조망권' 7만8506㎡ 부지에 세워진 영종자이는 114~322㎡형 아파트 1022가구로 이뤄졌다. 지상 10~24층 아파트 15개 동 규모다. 2006년 11월 분양 당시 시행사인 크레타건설과 시공사인 GS건설은 "영종자이는 서해바다 조망이 가능한 친환경적인 전원주택형 고급 아파트"라고 선전했다. 당시 3.3㎡당 분양가는 평균 1070만 원. 인근 영종하늘도시가 최근 분양을 하면서 3.3㎡당 분양가를 950만~980만 원에 내놓은 것에 비하면 분양가는 비쌌다. 그럼에도 부동산 가격 폭등기와 맞물려 1순위에서 청약이 마감되었고 계약도 2주 만에 모두 마쳤다. 입주 예정자들은 서해 조망권을 가진 명품 아파트에 들어설 날을 손꼽아 기다렸다. 오는 29일 입주 예정일을 앞두고 열린 지난달 사전 점검에서 GS건설

True

In [81]:
len(ad_df)

2182

In [143]:
s_model, s_tokenizer = get_model('psyche/KoT5-summarization', device)

psyche/KoT5-summarization
Using 2 GPUs.


In [156]:
summaries = []

for text in tqdm(ad_df.text):
    summary = summarize_long_sentence(text, 1024, s_tokenizer, s_model, device, max_tokens, min_tokens, num_beams)
    s_text = "".join(summary)
    summaries.append(s_text)
    
ad_df['summary'] = summaries

100%|█████████████████████████████████████| 2182/2182 [3:27:22<00:00,  5.70s/it]
/tmp/ipykernel_3056475/1918377928.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ad_df['summary'] = summaries


In [130]:
# final :
trues = 0
falses = 0
predict = []

for text in tqdm(ad_df.text):
    result = predict_ad(text, model, tokenizer)
    predict.append(result)
        
ad_df['predict_f'] = predict

100%|███████████████████████████████████████| 2182/2182 [00:43<00:00, 50.41it/s]
/tmp/ipykernel_3056475/3783739435.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ad_df['predict_f'] = predict


In [114]:
# 135701 :
trues = 0
falses = 0
predict = []

for text in tqdm(ad_df.text):
    result = predict_ad(text, model, tokenizer)
    predict.append(result)
        
ad_df['predict'] = predict

100%|███████████████████████████████████████| 2182/2182 [00:36<00:00, 59.30it/s]
/tmp/ipykernel_3056475/3830682088.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ad_df['predict'] = predict


In [165]:
# summary :
trues = 0
falses = 0
predict = []

for text in tqdm(ad_df.summary):
    result = predict_ad(text, model, tokenizer)
    predict.append(result)
        
ad_df['predict_s'] = predict

100%|███████████████████████████████████████| 2182/2182 [00:48<00:00, 44.63it/s]
/tmp/ipykernel_3056475/2573636431.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ad_df['predict_s'] = predict


In [168]:
ad_df[:10]

,id,topic,text,predict_f,predict,summary,predict_s
1406,NIRW1900000001.2436,경제,"고분양가 그대로 두고 미분양 해소! 바보 아냐? ""건설사들이 '같이 죽든지 아니면 ...",True,False,지난 2월 12일 정부가 미분양 해소를 위해 내놓은 양도소득세 면제·감면 조치의 효...,False
2245,NIRW1900000001.3840,경제,아파트 계약자들과 시공사 만남이 '화기애애'? 건설업체가 변하고 있다. 장기침체로 ...,True,False,"지난 3월 14일(토), 대전 도안신도시 16블록에 1253세대의 아파트가 시공되고...",False
2576,NIRW1900000001.4426,경제,도심 재개발 원주민 정든 마을 내쫓길 처지 재개발 보상가 문제로 일부 주민들의 거센...,True,False,재개발 보상가 문제로 일부 주민들의 거센 반발과 진통을 겪고 있는 안양시 만안구 안...,False
9456,NIRW1900000001.16085,경제,"""서해 조망권이라더니 민둥산 조망권이었나?"" ""강남에 지은 곳보다 건축비가 더 들었...",True,False,"2006년 11월 분양 당시 시행사인 크레타건설과 시공사인 GS건설은 ""영종자이는 ...",False
14437,NIRW1900000002.5420,경제,"대법, 분양률 높이려 과장 광고한 건설사 제동 신규 아파트 인근에 전철역이 들어선다...",True,False,현대산업개발(주)은 2002년 4월 경기도 파주시 교하읍에 건축될 '자유로 아이파크...,False
17327,NIRW1900000003.278,경제,"""값 비싼 불량 아파트... 살고 싶지 않다"" 아파트 모델하우스와 홍보책자만큼 한국...",False,True,"주택선분양제도 하에서 분양하는 우리나라 아파트의 경우, 건설사가 홍보한 내용과 다르...",False
62586,NIRW1900000012.20,경제,소형 평형(구25평형) 아파트 투자가 대세!!! 우미 건설은 영종 하늘 도시 내에 ...,False,True,우미 건설은 영종 하늘 도시 내에 1680가구를 일반 분양 중이다. 영종도 개발 계...,True
62596,NIRW1900000012.30,경제,청약통장 필요 없는 25평형 아파트 특별분양 “영종하늘도시 우미린” 우미 건설은 영...,False,True,영종 국제 도시 개발 호재로 투자 가치가 급상승중인천 경제 자유구역에서 영종 구역(...,True
62609,NIRW1900000012.45,경제,1500만원대로 2억대 신도시 아파트 잡으세요! -양도세가 얼마인데 100% 면제 ...,False,True,"영종도 개발 계획으로는 대형 의료단지 메디시티와 운북지구 복합 레저단지 조성, MG...",True
62666,NIRW1900000012.109,경제,"부동산 재테크 열기, 주목할 만한 곳은? 국내 실물경기 회복에 따라 올해 하반기 이...",True,False,국내 실물경기 회복에 따라 올해 하반기 이후부터는 부동산시장이 수도권을 중심으로 본...,True


In [132]:
ad_df.predict_f.value_counts()

False    1643
True      539
Name: predict_f, dtype: int64

In [133]:
ad_df.predict.value_counts()

True     1741
False     441
Name: predict, dtype: int64

In [166]:
ad_df.predict_s.value_counts()

True     1772
False     410
Name: predict_s, dtype: int64

In [170]:
ad_df2 = ad_df[ad_df['predict'] == False] 

In [171]:
ad_df2 = ad_df2[ad_df2['predict_s'] == True]

In [172]:
len(ad_df2)

126

In [173]:
ad_df2.iloc[0]

id                                          NIRW1900000012.109
topic                                                       경제
text         부동산 재테크 열기, 주목할 만한 곳은? 국내 실물경기 회복에 따라 올해 하반기 이...
predict_f                                                 True
predict                                                  False
summary      국내 실물경기 회복에 따라 올해 하반기 이후부터는 부동산시장이 수도권을 중심으로 본...
predict_s                                                 True
Name: 62666, dtype: object

In [174]:
ad_df2.iloc[0].text

'부동산 재테크 열기, 주목할 만한 곳은? 국내 실물경기 회복에 따라 올해 하반기 이후부터는 부동산시장이 수도권을 중심으로 본격적인 상승세를 나타낼 것으로 전망되고 있다. 다만 보금자리주택 공급확대와 준주택 도입, 위례신도시 분양, 지방선거 등 주요 현안들이 가시화 될 전망이어서 부동산 각 분야별로 상승 분위기는 차이가 있을 것으로 보인다. 그렇다면 올해 재테크 열기를 이어갈 주목할만한 부동산 투자처는 어디일까?- 2월 11일 세제 마감, 주목 받는 미분양 아파트는?교통호재와 개발호재가 있는 지역의 수도권 아파트 시장은 올해에도 꾸준한 인기를 끌 것으로 예상된다. 그 중에서도 2월 11일 정부의 세제 마감 시한을 앞둔 역세권 주변의 미분양 아파트들은 현재 마지막 수요가 몰리고 있다. 부동산전문가들은 할인혜택, 경품을 내건 미분양아파트에 혹하기보다는 향후 가치상승이 기대되는 곳에 분양 받을 것을 권하고 있다. ◇GS건설 서초 아트 자이GS건설이 서초동에 지은 서초아트자이를 특별 분양 하고 있다. 이 주상복합은 지하 4층, 지상 25층의 4개동으로 △181.10㎡(54평형)56가구 △207.49㎡(62평형)104가구 △336.71㎡(101평형) 4가구, 오피스텔 26가구 등 총 190가구 규모다. 서초 아트자이는 전용률이 80%이고 양면 개방형 신평면 구조를 채택해 통풍채광이 뛰어나고 전 가구가 방향이 다양하게 배치되었다. 3호선 남부터미널역과 2,3호선 교대역이 가까워 교통여건이 뛰어나며 서초 남부터미널 개발과, 롯데타운 개발예정에 따른 파급효과도 기대 할 수 있다. 또 서초아트자이는 주거지역, 중심업무지구, 문화예술공간이 공존하는 강남의 복합상권으로 시너지효과를 극대화할 수 있는 입지가 장점이다. 내부 주거 환경도 뛰어나 지상 22층 높이 스카이 브릿지에 170평 규모의 휘트니스센터와 게스트룸, 클럽하우스 등 자이안센터가 있으며 강남 시내와 관악산이 한눈에 보이는 자이만의 독특한 조망권을 느낄 수 있다. 6월 31일까지 취.등록세 감면된다. (분양문의:02-5

In [175]:
ad_df2.iloc[0].summary

'국내 실물경기 회복에 따라 올해 하반기 이후부터는 부동산시장이 수도권을 중심으로 본격적인 상승세를 나타낼 것으로 전망되고 있다. 다만 보금자리주택 공급확대와 준주택 도입, 위례신도시 분양, 지방선거 등 주요 현안들이 가시화 될 전망이어서 부동산 각 분야별로 상승 분위기는 차이가 있을 것으로 보인다. 교통호재와 개발호재가 있는 지역의 수도권 아파트 시장은 올해에도 꾸준한 인기를 끌 것으로 예상된다. 부동산전문가들은 할인혜택, 경품을 내건 미분양아파트에 혹하기보다는 향후 가치상승이 기대되는 곳에 분양 받을 것을 권하고 있다.우미린은 올림픽대로와 연결된 김포고속화도로 와 4번 국도 확장으로 자동차를 이용해 여의도까지 약 30분대 진입이 가능하며 단지에서 차량 약 10분 거리에 제2외곽순환도로가 연결되어 일산,청라, 송도까지 접근성도 우수하다. 총 14개 동으로 구성된 우미린의 계약시 조건은 계약금 5%, 중도금 전액 무이자(단, 일부 주택형별로 상이함)융자이며,선착순으로 수의계약으로 청약 통장 없이 계약이 가능하다.오피스텔에 대한 정부의 전폭적인 정책지원과 희소성이 부각되면서 역세권 등 위치가 좋고 임대수요가 많은 지역의 오피스텔은 활기를 띄고 있다. 부천시 중동에 분양중인 미니오피스텔 로하스 레지던스의 경우 모델 하우스 오픈 한 달 만에 이미 50%이상 계약을 완료했다. 주변에 많은 학원가가 밀집되어 있고 부천대학교, 부천시청, 홈플러스, GS백화점, 현대백화점 등이 가까이 있으며 주변으로 호텔과 높은 고층의 업무시설이 들어설 예정이어서 입지와 수요 면에서는 유리하다. 특히 내년 말 개통될 7호선 연장선(부천 중부경찰서역)을 이용하면 서울 강남까지 40분이면 출 퇴근 가능해 교통여건이 뛰어나다.'

In [148]:
summaries = summarize_long_sentence(text, 1024, s_tokenizer, s_model, device, max_tokens, min_tokens, num_beams)

In [149]:
summaries

['지난 6일 오후 판교원 9단지 한림풀에버와 도로를 사이에 두고 건너편에 위치한 11블록 단독주택단지 일대. 동화SFC하우징의 타운하우스 네이처하우스 빌리지 판교를 비롯해 주택건설 우리집과 판교단독주택가이드가 내놓은 모델하우스 등이 위치해 있다.판교 단독주택의 장점은 마당이나 정원을 낀 자신만의 쾌적한 공간을 가지면서 신도시 기반시설을 쉽게 이용할 수 있어 편리하다는 것. 특히 판교의 단독주택 거주자는 아파트 못지 않은 주거복지서비스를 보장받을 수 있을 전망이다.',
 '판교 단독택지는 작년까지만 해도 강남 등 서울에서 찾아오는 50~60대 장년층의 수요가 많았지만 올 봄 들어서는 강남 송파 분당 용인 수원은 물론 캐나다, 미국, 중국 등 해외교포들의 문의도 늘고 있다고. 판교 단독주택은 1대1 맞춤설계를 통해 주문자인 건축주의 취향이 100% 반영되는 것이 특징. 가족 구성원, 라이프스타일 등을 고려해 주택 설계팀과 충분한 상담을 거쳐 자신에게 가장 적합한 구조와 인테리어, 마감재를 선택하게 된다. 현재 판교 주요지역의 주거전용 택지는 평당 800~1000만원 대, 점포겸용주택지의 경우 이보다 높은 1000~1500만원에 거래된다.',
 '상가를 겸할 수 있는 판교의 점포겸용 주택지에 대해서도 관심이 뜨겁다. 특히 수도권 주택지 중에서도 판교신도시는 강남까지 20분 거리에 위치해 근접성 면에서 최고의 입지라는 게 전문가들의 공통된 의견이다. 특히 DTI적용을 받지 않기 때문에 80%까지 중도금 대출이 가능해 초기부담이 크지 않다는 것도 장점으로 꼽힌다. 특히 판교의 주거전용 단독택지가 1354개인데 반해 점포 겸용 택지는 602개에 불과해 희소성 면에서도 가치가 높다. 판교의 단독주택을 설계 시공 중인 주택건설 우리집의 조선호(54`건축사) 씨는 판교 건축주의 대다수가 집에서 가족애()를 찾고자 하는 40대 보보스 계층이라고 소개한다.',
 '판교에 공급되는 택지는 크게 이주자 택지와 협의양도인택지로 크게 두 종류. 판교에 주택을 소유하고 있는 원주민에게 

In [154]:
s_text = "".join(summaries)

In [155]:
s_text

'지난 6일 오후 판교원 9단지 한림풀에버와 도로를 사이에 두고 건너편에 위치한 11블록 단독주택단지 일대. 동화SFC하우징의 타운하우스 네이처하우스 빌리지 판교를 비롯해 주택건설 우리집과 판교단독주택가이드가 내놓은 모델하우스 등이 위치해 있다.판교 단독주택의 장점은 마당이나 정원을 낀 자신만의 쾌적한 공간을 가지면서 신도시 기반시설을 쉽게 이용할 수 있어 편리하다는 것. 특히 판교의 단독주택 거주자는 아파트 못지 않은 주거복지서비스를 보장받을 수 있을 전망이다.판교 단독택지는 작년까지만 해도 강남 등 서울에서 찾아오는 50~60대 장년층의 수요가 많았지만 올 봄 들어서는 강남 송파 분당 용인 수원은 물론 캐나다, 미국, 중국 등 해외교포들의 문의도 늘고 있다고. 판교 단독주택은 1대1 맞춤설계를 통해 주문자인 건축주의 취향이 100% 반영되는 것이 특징. 가족 구성원, 라이프스타일 등을 고려해 주택 설계팀과 충분한 상담을 거쳐 자신에게 가장 적합한 구조와 인테리어, 마감재를 선택하게 된다. 현재 판교 주요지역의 주거전용 택지는 평당 800~1000만원 대, 점포겸용주택지의 경우 이보다 높은 1000~1500만원에 거래된다.상가를 겸할 수 있는 판교의 점포겸용 주택지에 대해서도 관심이 뜨겁다. 특히 수도권 주택지 중에서도 판교신도시는 강남까지 20분 거리에 위치해 근접성 면에서 최고의 입지라는 게 전문가들의 공통된 의견이다. 특히 DTI적용을 받지 않기 때문에 80%까지 중도금 대출이 가능해 초기부담이 크지 않다는 것도 장점으로 꼽힌다. 특히 판교의 주거전용 단독택지가 1354개인데 반해 점포 겸용 택지는 602개에 불과해 희소성 면에서도 가치가 높다. 판교의 단독주택을 설계 시공 중인 주택건설 우리집의 조선호(54`건축사) 씨는 판교 건축주의 대다수가 집에서 가족애()를 찾고자 하는 40대 보보스 계층이라고 소개한다.판교에 공급되는 택지는 크게 이주자 택지와 협의양도인택지로 크게 두 종류. 판교에 주택을 소유하고 있는 원주민에게 주어지는 이주자택지(이택)는 

In [152]:
predict_ad(text, model, tokenizer)

False

In [153]:
predict_ad(merged_summaries, model, tokenizer)

False